In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# test cases to experiment
test = "467..114.."
test_2 = "...*......"
test_3 = "...%.$......"

In [10]:
# opening the file in read mode 
my_file = open("input.txt", "r") 
  
# reading the file 
data = my_file.read() 
  
# split on ('\n')
data_list = data.split("\n") 

In [11]:
# explore data
data_list[:3]

['.......12.......935............184.720...243........589.652..........435..........483.............6...........................904...........',
 '......*.....968*.....$............*........=..348...*..........986....*...................459....*........422................#......%482....',
 '....291............612....290..........903........699......218*.......376............890....*.838...81......*.....138.../194................']

In [12]:
# empty dict to fill
num_dict = {}
spec_char_dict = {}

In [13]:
# function to split based on any special char
def split_on_special_characters(input_string):
    pattern = re.compile(r'\W+')
    parts = pattern.split(input_string)
    return [part for part in parts if part!=""]

In [21]:
# loop over all rows, create special char and num dict to reference later on
for idx, row in enumerate(data_list):
    num_list = [item for item in split_on_special_characters(row) if item.isdigit()]
    spec_char_list = [item for item in row.split(".") if "*" in item]
    spec_char_list = [i for i in spec_char_list if i != '']
    num_list = list(set([''.join([i for i in item if i.isdigit()]) for item in num_list]))
    spec_char_list = list(set([''.join([i for i in item if not i.isalnum()]) for item in spec_char_list]))
    #combined_list = num_list + spec_char_list

    final_num_list = []
    # loop over each valid number in row, search for it as many times as it appears by
    # shortening the string each time you find the req string, then moving on when its no longer found
    for item in num_list:
        index = 0
        while index < len(row):
            #print(row)
            try:
                search_start = index + re.search('(?:\W|^)'+item+'(?:\W|$)',row[index:]).start()
                search_end = index + re.search('(?:\W|^)'+item+'(?:\W|$)',row[index:]).end()
                amended_item = row[search_start:search_end]
                if amended_item[0].isdigit():
                    start_idx = search_start
                    end_idx = search_end - 2
                elif amended_item[-1].isdigit():
                    start_idx = search_start + 1
                    end_idx = search_end - 1
                else:
                    start_idx = search_start + 1
                    end_idx = search_end - 2
                final_num_list.append([item, start_idx, end_idx])
                if index == -1:
                    break

                index = search_end + index
            except:
                index = len(row)
            
    #add list of lists for row idx, list of lists includes a list of each number with start and end index points, incl dup nums
    num_dict[idx] = final_num_list

    final_spec_char_list = []
    # loop over each valid spec char in row, search for it as many times as it appears by
    # shortening the string each time you find the req string, then moving on when its no longer found
    for item in spec_char_list:
        index = 0
        while index < len(row):
            index = row.find(item, index)
            if index == -1:
                break

            start_idx = index
            end_idx = start_idx + len(item) -1
            final_spec_char_list.append([item, start_idx, end_idx])
            index += 1
    #add list of lists for row idx, list of lists includes a list of each special char with start and end index points, incl dups
    spec_char_dict[idx] = final_spec_char_list

In [22]:
# review edge case
num_dict[104]

[['570', 53, 55],
 ['646', 13, 15],
 ['633', 91, 93],
 ['561', 2, 4],
 ['90', 130, 131],
 ['780', 137, 139],
 ['898', 39, 41],
 ['898', 111, 113],
 ['891', 124, 126],
 ['175', 49, 51],
 ['871', 102, 104],
 ['135', 74, 76],
 ['314', 96, 98]]

In [25]:
final_gears_list = []
validation_dict = {}
# loop over all rows and special chars, then compare to nums dict for adjacent and same row
# uses start and end numbers to detect whether theyre adjacent 
for row, gear_list in spec_char_dict.items():
    prev_row = row - 1
    next_row = row + 1

    #filters to only relevant three rows
    filtered_dict = {key: num_dict[key] for key in [prev_row, row, next_row] if key in num_dict}
    #print(row, filtered_dict)
    #print(part)
    # for each * compare to positions of all nums in relevant rows, add to list and keep track of how many its aligned to
    for gear in gear_list:
        num_aligned = 0
        num_list = []
        #print(row)
        #print(gear)
        for nums in filtered_dict.values():
            for num in nums:
                #print(num)
                # used to detect adjacent positions, num end compared to * start and end, same for num start
                if ((num[2] >= gear[1]-1) & (num[2] <= gear[2]+1)) | ((num[1] >= gear[1]-1) & (num[1] <= gear[2]+1)):
                    num_aligned += 1
                    num_list.append(int(num[0]))
        #print(num_aligned)
        # creates dict for each row and position to show adjacent numbers to validate
        validation_dict[f"{row}_{gear[0]}_{gear[1]}"] = num_list
        # add the product of all in list if its 2 or more
        if num_aligned >= 2:
            #print(row, num_list, np.prod(num_list))
            final_gears_list.append(np.prod(num_list))

In [27]:
sum(final_gears_list)

84584891

In [26]:
validation_dict

{'1_*_6': [12, 291],
 '1_*_15': [935, 968],
 '1_*_34': [720, 184],
 '1_*_52': [589, 699],
 '1_*_70': [435, 376],
 '1_*_97': [6, 838],
 '2_*_62': [986, 218],
 '2_*_92': [459, 227],
 '2_*_108': [422, 225],
 '3_*_26': [290, 591],
 '3_*_99': [81, 533],
 '3_*_133': [424, 603],
 '4_*_16': [156, 938],
 '4_*_29': [891, 470],
 '4_*_46': [89, 746],
 '5_*_6': [229, 952],
 '5_*_61': [107, 867],
 '5_*_116': [909, 788],
 '6_*_35': [555, 892],
 '6_*_109': [826, 709],
 '7_*_73': [102, 491],
 '7_*_84': [542, 779],
 '7_*_113': [8, 974],
 '7_*_128': [511, 270],
 '8_*_19': [237, 53],
 '8_*_27': [171, 781],
 '9_*_95': [877, 465],
 '9_*_118': [359, 192],
 '10_*_6': [342, 468],
 '10_*_43': [336, 789],
 '10_*_93': [621],
 '10_*_123': [910],
 '11_*_22': [573, 905],
 '11_*_53': [256, 524],
 '12_*_72': [677, 955],
 '12_*_87': [62, 257],
 '12_*_128': [297, 545],
 '13_*_10': [69, 668],
 '13_*_67': [127, 66],
 '13_*_78': [714, 660],
 '15_*_28': [840, 460],
 '15_*_134': [25, 936],
 '17_*_53': [146, 995],
 '17_*_59':